In [23]:
import pandas as pd 

product_matrix = pd.read_csv("../data/raw/product_matrix.csv")

product_matrix.head(3)
pd.set_option("display.max_colwidth", None)


In [24]:
'''Normalization and splitting fucntions'''

import re 

def normalize_text(s):
    if pd.isna(s):
        return ""
    s = str(s).lower()
    s=re.sub(r"\s+", " ", s)
    return s.strip()

import re

def split_products(s):
    if not s:
        return []
    
    # 1) insert a separator AFTER "(xN)" if another letter starts right away
    #    e.g. "... (x1)rurka ..." -> "... (x1) | rurka ..."
    s = re.sub(r"(\(x\d+\))(?=\S)", r"\1 | ", s)

    # 2) now split on " + ", on "zestaw" starts, and on our inserted separator
    parts = re.split(r"\s\+\s|\s\|\s|(?=\bzestaw\b)", s)

    return [p.strip() for p in parts if p.strip()]

STOP_TOKENS = {"outlet"}  # can expand later if needed

def split_products_matrix(s):
    parts = split_products(s)  # reuse your existing splitter

    cleaned = []
    for p in parts:
        p = p.strip()

        # drop pure tokens like "outlet"
        if p in STOP_TOKENS:
            continue

        # if something still starts with "outlet ", strip it
        p = re.sub(r"^outlet\s+", "", p).strip()

        if p:
            cleaned.append(p)

    return cleaned

In [25]:
import pandas as pd
import numpy as np

# 1) drop junk columns like "Unnamed: 5"
junk_cols = [c for c in product_matrix.columns if str(c).startswith("Unnamed")]
product_matrix = product_matrix.drop(columns=junk_cols)

# 2) normalize key string columns
product_matrix["produkty_clean"] = product_matrix["Produkty"].apply(normalize_text)

for col in ["RODZAJ", "BAZOWY", "CZY JEST W MATRIXIE", "MATRIX NAZWA", "MATRIX GRUPA PRODUKTOWA"]:
    if col in product_matrix.columns:
        product_matrix[col] = product_matrix[col].astype(str).str.strip()

# 3) numeric: filters count (and "Ilość" if you end up using it)
product_matrix["ILOŚĆ FILTRÓW"] = pd.to_numeric(product_matrix["ILOŚĆ FILTRÓW"], errors="coerce")

product_matrix.head()


,Produkty,RODZAJ,ILOŚĆ FILTRÓW,BAZOWY,Ilość,CZY JEST W MATRIXIE,MATRIX NAZWA,MATRIX GRUPA PRODUKTOWA,produkty_clean
0,BARBIE 3 filtry do butelki filtrującej Dafi SOFT i SOLID Różowy z naklejkami,BUTELKOWE,3.0,nan,NaN,TAK,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,barbie 3 filtry do butelki filtrującej dafi soft i solid różowy z naklejkami
1,OUTLET ZESTAW 2 filtry + nakrętka do butelki filtrującej Dafi SOFT cytrynowy,BUTELKOWE,2.0,nan,NaN,TAK,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,outlet zestaw 2 filtry + nakrętka do butelki filtrującej dafi soft cytrynowy
2,OUTLET ZESTAW 2 filtry + nakrętka do butelki filtrującej Dafi SOFT czarny,BUTELKOWE,2.0,nan,NaN,TAK,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,outlet zestaw 2 filtry + nakrętka do butelki filtrującej dafi soft czarny
3,OUTLET ZESTAW 2 filtry + nakrętka do butelki filtrującej Dafi SOFT flamingowy,BUTELKOWE,2.0,nan,NaN,TAK,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,outlet zestaw 2 filtry + nakrętka do butelki filtrującej dafi soft flamingowy
4,OUTLET ZESTAW 2 filtry + nakrętka do butelki filtrującej Dafi SOFT jagodowy,BUTELKOWE,2.0,nan,NaN,TAK,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,outlet zestaw 2 filtry + nakrętka do butelki filtrującej dafi soft jagodowy


In [26]:
ATTR_COLS = [
    "RODZAJ",
    "ILOŚĆ FILTRÓW",
    "BAZOWY",
    "MATRIX NAZWA",
    "MATRIX GRUPA PRODUKTOWA",
    "CZY JEST W MATRIXIE",
]

sku_map_df = (
    product_matrix
    .drop_duplicates(subset=["produkty_clean"])
    [["produkty_clean", "Produkty"] + ATTR_COLS]
    .reset_index(drop=True)
)

print("Matrix rows:", len(product_matrix))
print("Unique SKUs:", len(sku_map_df))
sku_map_df.head()


Matrix rows: 1128
Unique SKUs: 1112


,produkty_clean,Produkty,RODZAJ,ILOŚĆ FILTRÓW,BAZOWY,MATRIX NAZWA,MATRIX GRUPA PRODUKTOWA,CZY JEST W MATRIXIE
0,barbie 3 filtry do butelki filtrującej dafi soft i solid różowy z naklejkami,BARBIE 3 filtry do butelki filtrującej Dafi SOFT i SOLID Różowy z naklejkami,BUTELKOWE,3.0,nan,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,TAK
1,outlet zestaw 2 filtry + nakrętka do butelki filtrującej dafi soft cytrynowy,OUTLET ZESTAW 2 filtry + nakrętka do butelki filtrującej Dafi SOFT cytrynowy,BUTELKOWE,2.0,nan,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,TAK
2,outlet zestaw 2 filtry + nakrętka do butelki filtrującej dafi soft czarny,OUTLET ZESTAW 2 filtry + nakrętka do butelki filtrującej Dafi SOFT czarny,BUTELKOWE,2.0,nan,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,TAK
3,outlet zestaw 2 filtry + nakrętka do butelki filtrującej dafi soft flamingowy,OUTLET ZESTAW 2 filtry + nakrętka do butelki filtrującej Dafi SOFT flamingowy,BUTELKOWE,2.0,nan,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,TAK
4,outlet zestaw 2 filtry + nakrętka do butelki filtrującej dafi soft jagodowy,OUTLET ZESTAW 2 filtry + nakrętka do butelki filtrującej Dafi SOFT jagodowy,BUTELKOWE,2.0,nan,1 FILTR BUTELKOWY,06_filtry do butelek Soft i Solid,TAK


In [27]:
dup_keys = product_matrix[product_matrix.duplicated("produkty_clean", keep=False)].copy()
print("Rows with duplicate produkty_clean:", len(dup_keys))

if len(dup_keys) > 0:
    conflict_check = dup_keys.groupby("produkty_clean")[ATTR_COLS].nunique()
    conflict_keys = conflict_check[(conflict_check > 1).any(axis=1)].index
    print("Duplicate keys with conflicting attributes:", len(conflict_keys))

    if len(conflict_keys) > 0:
        display(
            dup_keys[dup_keys["produkty_clean"].isin(conflict_keys)]
            .sort_values("produkty_clean")[["produkty_clean", "Produkty"] + ATTR_COLS]
            .head(50)
        )


Rows with duplicate produkty_clean: 32
Duplicate keys with conflicting attributes: 0


## Build SKU Lookup & Export Artifacts

This step converts the **product matrix** into two clean, reusable artifacts that act as the
single source of truth for product decoding in downstream notebooks.

### What this code does

1. **Creates a canonical SKU table**
   - Uses `produkty_clean` (normalized product name) as the SKU key
   - Deduplicates rows safely (verified earlier: no attribute conflicts)
   - Retains all relevant product attributes:
     - product type (`RODZAJ`)
     - number of filters included (`ILOŚĆ FILTRÓW`)
     - base product (`BAZOWY`)
     - matrix name and product group
     - matrix inclusion flag

2. **Builds a fast lookup dictionary**
   - Converts the SKU table into a Python dict:
     ```
     produkty_clean → product attributes
     ```
   - This structure is optimized for fast runtime decoding of order strings

3. **Exports artifacts for downstream use**
   - `sku_map.parquet`
     - Tabular, inspectable representation of the product matrix
     - Used for audits, joins, and analytical exploration
   - `sku_lookup.json`
     - Lightweight key–value lookup used during order decoding
     - Language-agnostic and easy to load at runtime

### Why this matters

Separating the product matrix into a **data artifact (Parquet)** and a **runtime lookup (JSON)**
allows the pipeline to be:
- fast and scalable
- easy to debug and audit
- reusable across notebooks and future applications

Downstream notebooks (e.g. order decoding, retention modeling, LTV prediction) rely exclusively
on these exported artifacts, ensuring consistent product interpretation across the entire pipeline.


In [28]:
# 1 row per SKU (safe because no attribute conflicts)
sku_map_df = (
    product_matrix
    .drop_duplicates(subset=["produkty_clean"])
    [["produkty_clean", "Produkty"] + ATTR_COLS]
    .reset_index(drop=True)
)

print("Unique SKUs:", len(sku_map_df))

# dict for fast lookup during order decoding
sku_lookup = sku_map_df.set_index("produkty_clean")[ATTR_COLS].to_dict("index")

# Save artifacts for downstream notebooks
import os, json
os.makedirs("../data/interim", exist_ok=True)

sku_map_df.to_parquet("../data/interim/sku_map.parquet", index=False)

with open("../data/interim/sku_lookup.json", "w", encoding="utf-8") as f:
    json.dump(sku_lookup, f, ensure_ascii=False, indent=2)

print("Saved:")
print(" - data/interim/sku_map.parquet")
print(" - data/interim/sku_lookup.json")


Unique SKUs: 1112
Saved:
 - data/interim/sku_map.parquet
 - data/interim/sku_lookup.json


In [29]:
# check a few keys
for k in list(sku_lookup.keys())[:5]:
    print(k, "->", sku_lookup[k])


barbie 3 filtry do butelki filtrującej dafi soft i solid różowy z naklejkami -> {'RODZAJ': 'BUTELKOWE', 'ILOŚĆ FILTRÓW': 3.0, 'BAZOWY': 'nan', 'MATRIX NAZWA': '1 FILTR BUTELKOWY', 'MATRIX GRUPA PRODUKTOWA': '06_filtry do butelek Soft i Solid', 'CZY JEST W MATRIXIE': 'TAK'}
outlet zestaw 2 filtry + nakrętka do butelki filtrującej dafi soft cytrynowy -> {'RODZAJ': 'BUTELKOWE', 'ILOŚĆ FILTRÓW': 2.0, 'BAZOWY': 'nan', 'MATRIX NAZWA': '1 FILTR BUTELKOWY', 'MATRIX GRUPA PRODUKTOWA': '06_filtry do butelek Soft i Solid', 'CZY JEST W MATRIXIE': 'TAK'}
outlet zestaw 2 filtry + nakrętka do butelki filtrującej dafi soft czarny -> {'RODZAJ': 'BUTELKOWE', 'ILOŚĆ FILTRÓW': 2.0, 'BAZOWY': 'nan', 'MATRIX NAZWA': '1 FILTR BUTELKOWY', 'MATRIX GRUPA PRODUKTOWA': '06_filtry do butelek Soft i Solid', 'CZY JEST W MATRIXIE': 'TAK'}
outlet zestaw 2 filtry + nakrętka do butelki filtrującej dafi soft flamingowy -> {'RODZAJ': 'BUTELKOWE', 'ILOŚĆ FILTRÓW': 2.0, 'BAZOWY': 'nan', 'MATRIX NAZWA': '1 FILTR BUTELKOWY', 